<a href="https://colab.research.google.com/github/usunn/han_AI/blob/main/pill_YOLOv8_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive, userdata
drive.mount('/content/drive')        # ↲ Google Drive 연결
!pip install -q ultralytics         # ↲ YOLOv8 최신판


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 879.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.3 MB/s eta 0:00:00


In [ ]:
import os, shutil, zipfile, glob

# (1) Google-Drive 안 원본 zip 두 개가 있는 폴더
base_drive = '/content/drive/MyDrive/166.약품식별 인공지능 개발을 위한 경구약제 이미지 데이터/01.데이터/1.Training'
label_zip = os.path.join(base_drive, '라벨링데이터/경구약제조합 5000종/TL_1_조합.zip')
image_zip = os.path.join(base_drive, '원천데이터/경구약제조합 5000종/TS_1_조합.zip')

# (2) Colab 로컬에 모든 작업을 넣을 루트
root = '/content/pill_dataset'      # -- 알아서 변경 가능

# 깨끗하게 비우고 다시 만들기
if os.path.exists(root): shutil.rmtree(root)
os.makedirs(root, exist_ok=True)

ori_label_dir = os.path.join(root, 'original/label')
ori_img_dir   = os.path.join(root, 'original/image')
os.makedirs(ori_label_dir), os.makedirs(ori_img_dir)


(None, None)

In [ ]:
# ↓ 압축 해제는 zipfile 모듈이 훨씬 빠릅니다
for zfile, outdir in [(label_zip, ori_label_dir), (image_zip, ori_img_dir)]:
    with zipfile.ZipFile(zfile) as z:
        z.extractall(outdir)
print('✅ 압축 해제 완료')


✅ 압축 해제 완료


In [ ]:
import os, json, glob, shutil, tqdm, pandas as pd
from sklearn.model_selection import train_test_split

# ----------------- 환경 설정 -----------------
root          = '/content'                   # 코랩 작업 루트
ori_label_dir = f'{root}/pill_dataset/original/label'
ori_img_dir   = f'{root}/pill_dataset/original/image'
yolo_root     = f'{root}/pill_dataset/yolo_dataset'
debug         = True                         # ← False 로 바꾸면 로그 최소화

# 결과 폴더 준비
for sub in ['images/train', 'images/val', 'labels/train', 'labels/val']:
    os.makedirs(os.path.join(yolo_root, sub), exist_ok=True)

# ----------------- 1. JSON 순회 -----------------
json_paths = glob.glob(f'{ori_label_dir}/**/*.json', recursive=True)
good, skipped = [], []

for jpath in tqdm.tqdm(json_paths, desc='convert'):
    try:
        data = json.load(open(jpath, encoding='utf-8'))
        if not data.get('images'):                                   # 필수 필드 체크
            raise ValueError('images-key-missing')

        img_name = data['images'][0]['file_name']                    # PNG 파일명
        # ① 1차: JSON과 같은 상위 폴더 이름으로 조합
        folder   = os.path.basename(os.path.dirname(jpath))
        img_src  = os.path.join(ori_img_dir, folder, img_name)

        # ② 2차: 전역 검색(특수 폴더 구조 대응)
        if not os.path.exists(img_src):
            matches = glob.glob(f'{ori_img_dir}/**/{img_name}', recursive=True)
            if matches: img_src = matches[0]

        if not os.path.exists(img_src):
            raise FileNotFoundError('image-missing')

        w, h = data['images'][0]['width'], data['images'][0]['height']
        ytxt_lines = []
        for ann in data.get('annotations', []):
            bbox = ann.get('bbox', [])
            if len(bbox) != 4:
                raise ValueError(f'bbox-len-{len(bbox)}')
            x, y, bw, bh = bbox
            xc, yc = (x + bw/2)/w, (y + bh/2)/h
            ytxt_lines.append(
                f"{ann['category_id']-1} {xc:.6f} {yc:.6f} {bw/w:.6f} {bh/h:.6f}"
            )

        if not ytxt_lines:
            raise ValueError('empty-annotation')

        good.append((img_src, '\n'.join(ytxt_lines)))

    except Exception as e:
        skipped.append((jpath, str(e)))
        if debug:
            print(f'⛔ {e} -> {jpath}')

# ----------------- 2. train/val 분리 -----------------
if not good:
    raise RuntimeError('❗ 변환된 샘플이 0개입니다. 상단 로그/skip CSV를 확인하세요.')

train, val = train_test_split(good, test_size=0.1, random_state=42)

def dump(pairs, split):
    """이미지 복사 + txt 저장"""
    for img_src, ytxt in tqdm.tqdm(pairs, desc=split):
        img_dst = os.path.join(yolo_root, f'images/{split}', os.path.basename(img_src))
        txt_dst = os.path.join(
            yolo_root, f'labels/{split}', os.path.basename(img_src).replace('.png', '.txt')
        )
        shutil.copy(img_src, img_dst)
        with open(txt_dst, 'w') as f: f.write(ytxt)

dump(train, 'train')
dump(val, 'val')

print(f"\n✅ 최종 결과: train {len(train)} / val {len(val)}  (skip {len(skipped)})")

# ----------------- 3. 스킵 목록 CSV -----------------
pd.DataFrame(skipped, columns=['json_path', 'reason']).to_csv('skipped_info.csv', index=False)
print("📄 skipped_info.csv 저장 완료")


convert:   3%|▎         | 163/6011 [00:04<03:32, 27.48it/s]

⛔ bbox-len-3 -> /content/pill_dataset/original/label/K-000250-002483-022362-022627_json/K-022627/K-000250-002483-022362-022627_0_2_0_2_90_000_200.json


convert:   9%|▊         | 521/6011 [00:09<01:00, 90.44it/s]

⛔ bbox-len-0 -> /content/pill_dataset/original/label/K-000250-002483-006192-022362_json/K-000250/K-000250-002483-006192-022362_0_2_0_2_70_000_200.json


convert:  19%|█▊        | 1122/6011 [00:16<00:52, 93.18it/s]

...

⛔ bbox-len-0 -> /content/pill_dataset/original/label/K-000250-002483-012778-013395_json/K-002483/K-000250-002483-012778-013395_0_2_0_2_90_000_200.json


convert:  94%|█████████▍| 5663/6011 [01:09<00:03, 98.96it/s]

⛔ bbox-len-0 -> /content/pill_dataset/original/label/K-001866-004378-005002-022627_json/K-004378/K-001866-004378-005002-022627_0_2_0_2_70_000_200.json


val: 100%|██████████| 600/600 [00:05<00:00, 104.26it/s]



✅ 최종 결과: train 5391 / val 600  (skip 20)
📄 skipped_info.csv 저장 완료


In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')        # 경량 → 빠른 프로토타입
model.train(
    data='/content/pill_dataset/yolo_dataset/data.yaml',
    epochs=50, imgsz=640,
    batch=16,                     # VRAM 여유 없으면 8-4 로 축소
    name='pill_v1'
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 111MB/s]


Ultralytics 8.3.166 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/pill_dataset/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pill_v1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pos

100%|██████████| 755k/755k [00:00<00:00, 19.8MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all        508        508      0.249      0.994      0.263      0.259



50 epochs completed in 1.046 hours.
Optimizer stripped from runs/detect/pill_v1/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/pill_v1/weights/best.pt, 6.2MB

Validating runs/detect/pill_v1/weights/best.pt...
Ultralytics 8.3.166 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:16<00:00,  1.06s/it]


                   all        508        508      0.248      0.994      0.286       0.28
Speed: 0.2ms preprocess, 2.2ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs/detect/pill_v1


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ebb27194d50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 